# GeoPythonConf2021

## eemont: A Python package that extends Google Earth Engine.
### David Montero Loaiza

#### How things work in the GEE Python API?

Import the Landsat 8 SR product:

In [ ]:
import ee

ee.Authenticate()
ee.Initialize()

L8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

Define a cloud mask function to map it over the collection:

In [ ]:
import ee

ee.Authenticate()
ee.Initialize()

def maskClouds(image):
  cloudShadowBitMask = (1 << 3)
  cloudsBitMask = (1 << 5)
  qa = image.select('pixel_qa')
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask)

L8 = (ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
      .map(maskClouds())

If you want to scale the collection, you also have to define a function for that:

In [ ]:
import ee

ee.Authenticate()
ee.Initialize()

def maskClouds(image):
  cloudShadowBitMask = (1 << 3)
  cloudsBitMask = (1 << 5)
  qa = image.select('pixel_qa')
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask)

def scaleImage(image):
  scaled = image.select('B[1-7]').multiply(0.0001)
  scaled = scaled.addBands(image.select(['B10','B11']).multiply(0.1))
  scaled = scaled.addBands(image.select(['sr_aerosol','pixel_qa','radsat_qa']))
  return scaled.copyProperties(image,image.propertyNames())

L8 = (ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
      .map(maskClouds)
      .map(scaleImage))

What if you want to compute some Spectral Indices? Well...

In [ ]:
import ee

ee.Authenticate()
ee.Initialize()

def maskClouds(image):
  cloudShadowBitMask = (1 << 3)
  cloudsBitMask = (1 << 5)
  qa = image.select('pixel_qa')
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask)

def scaleImage(image):
  scaled = image.select('B[1-7]').multiply(0.0001)
  scaled = scaled.addBands(image.select(['B10','B11']).multiply(0.1))
  scaled = scaled.addBands(image.select(['sr_aerosol','pixel_qa','radsat_qa']))
  return scaled.copyProperties(image,image.propertyNames())

def addIndices(image):
  NDVI = image.normalizedDifference(['B5','B4']).rename('NDVI')
  EVI = image.expression('2.5 * (b("B5") - b("B4")) / (b("B5") + 6.0 * b("B4") - 7.5 * b("B2") + 1.0)').rename('EVI')
  GNDVI = image.normalizedDifference(['B5','B3']).rename('GNDVI')
  return image.addBands([NDVI,EVI,GNDVI])

L8 = (ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
      .map(maskClouds)
      .map(scale)
      .map(addIndices))

#### How things work with eemont?

Well, you just need 9 lines of code to do the exact same previous processing!

In [ ]:
import ee, eemont

ee.Authenticate()
ee.Initialize()

L8 = (ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
      .maskClouds()
      .scale()
      .index(['NDVI','EVI','GNDVI']))

#### Features:

- Closest image to a specific date:

In [ ]:
import ee, eemont

ee.Authenticate()
ee.Initialize()

poi = ee.Geometry.Point([-76.4,3.21])

L8 = (ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
      .filterBounds(poi)
      .closest('2021-10-15'))

- Masking clouds and shadows:

In [ ]:
import ee, eemont

ee.Authenticate()
ee.Initialize()

poi = ee.Geometry.Point([-76.4,3.21])

L8 = (ee.ImageCollection('COPERNICUS/S2_SR')
      .filterBounds(poi)
      .maskClouds(prob = 60,
                  maskCirrus = False,
                  buffer = 300,
                  cdi = -0.5))

- Scaling and offsetting:

In [ ]:
import ee, eemont

ee.Authenticate()
ee.Initialize()

poi = ee.Geometry.Point([-76.4,3.21])

L8 = (ee.ImageCollection('MODIS/006/MOD11A2')
      .filterBounds(poi)
      .scale())

- Computing Spectral Indices:

In [ ]:
import ee, eemont

ee.Authenticate()
ee.Initialize()

poi = ee.Geometry.Point([-76.4,3.21])

L8 = (ee.ImageCollection('MODIS/006/MOD09GQ')
      .filterBounds(poi)
      .scale()
      .index(['NDVI','EVI2','kNDVI']))

- Time series by region or regions:

In [ ]:
import ee, eemont

ee.Authenticate()
ee.Initialize()

pivots = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point([27.724856,26.485040]).buffer(400),{'pivot':0}),
    ee.Feature(ee.Geometry.Point([27.719427,26.478505]).buffer(400),{'pivot':1}),
    ee.Feature(ee.Geometry.Point([27.714185,26.471802]).buffer(400),{'pivot':2})    
])

L8 = (ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
      .filterBounds(pivots)
      .maskClouds()
      .scale()
      .index(['EVI','GNDVI']))

ts = L8.getTimeSeriesByRegions(collection = pivots,
                               bands = ['EVI','GNDVI'],
                               reducer = [ee.Reducer.mean(),ee.Reducer.median()],
                               scale = 30)

- Overloaded operators:

In [ ]:
import ee, eemont

ee.Authenticate()
ee.Initialize()

point = ee.Geometry.Point([-76.0269,2.92846])

S2 = (ee.ImageCollection('COPERNICUS/S2_SR')
      .filterBounds(point)
      .sort('CLOUDY_PIXEL_PERCENTAGE')
      .first()
      .maskClouds()
      .scale()
      .index('NDSI'))

NDSI = S2.select('NDSI')
N = S2.select('B8')
G = S2.select('B3')

snowPixels = (NDSI > 0.4) & (N >= 0.1) & (G > 0.11)

- Constructors by queries (geocoding):

In [ ]:
import ee, eemont

ee.Authenticate()
ee.Initialize()

user_agent = 'eemont-geopythonconf-2021'

point = ee.Geometry.PointFromQuery('Cali, Colombia',user_agent = user_agent)
bbox = ee.Feature.BBoxFromQuery('Germany',user_agent = user_agent)
multipoint = ee.FeatureCollection.MultiPointFromQuery('Amazonas',user_agent = user_agent)